In [1]:
import sys
sys.path.append('.')
import pytorch_lightning as pl
import torch
from src import data_loaders
from src.models import resnet_18
from scripts.eval_resnet18 import eval
from scripts.zfp_compress import zfp_compress

device = torch.device('cpu')

In [ ]:
original_model = resnet_18.ResNetLightning()
ckpt = torch.load('models/resnet18_base_best-v1.ckpt', device=torch.device('gpu'))
original_model.load_state_dict(ckpt)
original_model.eval()

In [ ]:
tolerance_values = [1/2**i for i in range(-2, 27)]
results = list()
test_loader = data_loaders.cifar10_loader(train=False, num_workers=4)

accuracies = []
compression_ratios = []
model_sizes = []

for tolerance in tolerance_values:
    zfp_model, org_size, comp_size = zfp_compress(original_model, tolerance=tolerance)
    trainer_zfp = pl.Trainer()
    trials = trainer_zfp.test(zfp_model, dataloaders=test_loader)

    #Metrics
    accuracies.append(trials[0]['test_acc'])
    compression_ratios.append(org_size / comp_size)
    model_sizes.append(comp_size / 1024**2) #MB size



In [16]:
# torch.save({'state_dict' : zfp_model.state_dict()}, "models/zfp_resnet18.ckpt")